In [1]:
#***********************************#
#                                   #
#             Packages              #        
#                                   #
#***********************************#

import csv
import allantools as allan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import allantools as AT

In [2]:
#***********************************#
#                                   #
#             Constants             #        
#                                   #
#***********************************#

adc_clk = 512 * 10 ** 6 # adc clock rate

T_adc = 1/(adc_clk) # adc period

fpga_clk = 256 * 10 ** 6 # fpga fabric clock rate

FFT_length = 1024 # Length of FFT used in spectrometer

accum_len = 2 ** 23 # number of clock cycles of accumulation in every dump of data

#run_time = 100 # total amount of time that data will be simulated and collected

accum_time = ((adc_clk)/FFT_length)/accum_len # amount of time one accumulation cycle integrates (seconds)

#num_accum = run_time * (1/accum_time) # number of total accumulations in simulation time

# for testing, consider a 100 MHz source
source_freq = 100 * 10 ** 6 
square_freq = 100 * 10 ** 3


###########################
#     Make FFT Frames     #
###########################

    ##### for a 1024 pt FFT, one FFT frame will take 1024/adc_clk (seconds) ##### 

# frame time = Time for 1 FFT to populate
frame_time = FFT_length / (adc_clk)

# frame_freq = how many FFT frames are created in one second
frame_freq = adc_clk/FFT_length

##### The number of fft frames to be created equals the                        #
    #     run time of the test over the amount of time it takes to FFT a frame #####

# accum_frames = number of FFT frames created over one accumulation

accum_frames = int(accum_time/frame_time)

# Now create array which contains frequency span of FFT in order to have correctly scaled FFT plots
timestep = 1/adc_clk
# note this value comes from the linspace used for the time array. length/number of samples
fft_freq = np.fft.fftfreq(FFT_length, d=timestep)


In [3]:
#***********************************#
#                                   #
#            Functions              #        
#                                   #
#***********************************#

def real_wave(amp, freq, time):
    omega = 2 * np.pi * freq    
    wave = amp*np.cos(omega * time)
    return wave

def real_mix(wave_1, wave_2):
    mix_wave = wave_1 * wave_2
    return mix_wave
    
def fft(signal):
    spectrum = np.fft.fft(signal, n = FFT_length)
    return spectrum

def noisify(signal):
    w_noise = np.random.normal(0, .1, signal.shape)
    noisy_spec = signal * w_noise
    return noisy_spec

def GET_TO_DA_CHOPPAH(signal, timespace):
    sq_wave = 0.5 * (sig.square(2 * np.pi * square_freq * timespace) + 1)
    chopped_wave = real_mix(signal, sq_wave)
    return chopped_wave

In [4]:
#***********************************#
#                                   #
#           Creating Time           #        
#                                   #
#***********************************#

frame_times = np.zeros(accum_frames)

##### Create an array with the times of the FFT frames #####

for i in range (accum_frames):                                 
    
    frame_times[i] = i * frame_time
    
    ##### Create an array of times that will be used to create the "pieces" of the wave #####

timespace = np.zeros((accum_frames-1, FFT_length))

    ##### Populate time array with lengths to be used later #####

for i in range(accum_frames-1):
    
    timespace[i] = np.linspace(frame_times[i], frame_times[i+1], FFT_length)

    ##### Make an array to store FFT frames #####

fft_frames = np.zeros((accum_frames, FFT_length), dtype = complex)

In [7]:
## Simulating the hoh_spec firmware ##

accum_len = 2 ** 23 

def hoh_spec(chan, run_time):
        
        # Define how many seconds of data should be simulated and how many accumulations are required
    
    num_accum = int(run_time * (1/accum_time)) # number of total accumulations in simulation time
    
        # Open file for data to be written to
    
    file = open('sim_spec_data_%d.csv'%(chan), 'w')
    writer = csv.writer(file)
    
        #Make an empty array to be filled with accumulations
        
    accums = np.zeros(num_accum)
    
    #***********************************#
    #                                   #
    #           Creating Time           #        
    #                                   #
    #***********************************#
    
    for i in range(num_accum-1):
        print('we are on accumulation number %d out of %d'%(i, num_accum))
        
        frame_times = np.zeros(accum_frames)

            ##### Create an array with the times of the FFT frames #####

        for j in range(accum_frames):                                 
            
            frame_times[j] = (i * frame_time) + j * frame_time

            ##### Create an array of times that will be used to create the "pieces" of the wave #####

        timespace = np.zeros((accum_frames-1, FFT_length))

            ##### Populate time array with lengths to be used later #####

        for j in range(accum_frames-1):

            timespace[j] = np.linspace(frame_times[j], frame_times[j+1], FFT_length)

            ##### Make an array to store FFT frames #####

        # fft_frames = np.zeros((accum_frames, FFT_length), dtype = complex)

            # create white noise equivalent to what is used in tests
        
        signal = np.zeros(np.shape(timespace))
        for j in range(accum_frames-1):
            signal[j] = allan.noise.white(FFT_length, 0.1, adc_clk) #create large array of white noise

            # Now put the noise signal through PFB
        
        spectra = fft(signal)

        #t_streams = np.zeros((FFT_length, accum_frames-1), dtype=complex)
        #print(np.shape(t_streams))

            ##### Slice and dice array to convert frames into time streams #####

        t_streams = np.transpose(spectra)

            # Now take the intensity of each of the time streams

        t_mags = np.square(np.real(t_streams)) + np.square(np.imag(t_streams))
        
            # Write the timestream of the channel in question to a CSV file
        
        accum = np.sum(t_mags[chan])
        
        accums[i] = accum
    
    file.close()
    
    return accums
    





In [8]:
accum_data = hoh_spec(100, 100)

we are on accumulation number 0 out of 1677
we are on accumulation number 1 out of 1677
we are on accumulation number 2 out of 1677
we are on accumulation number 3 out of 1677
we are on accumulation number 4 out of 1677
we are on accumulation number 5 out of 1677
we are on accumulation number 6 out of 1677
we are on accumulation number 7 out of 1677
we are on accumulation number 8 out of 1677
we are on accumulation number 9 out of 1677
we are on accumulation number 10 out of 1677
we are on accumulation number 11 out of 1677
we are on accumulation number 12 out of 1677
we are on accumulation number 13 out of 1677
we are on accumulation number 14 out of 1677
we are on accumulation number 15 out of 1677
we are on accumulation number 16 out of 1677
we are on accumulation number 17 out of 1677
we are on accumulation number 18 out of 1677
we are on accumulation number 19 out of 1677
we are on accumulation number 20 out of 1677
we are on accumulation number 21 out of 1677
we are on accumulati

we are on accumulation number 181 out of 1677
we are on accumulation number 182 out of 1677
we are on accumulation number 183 out of 1677
we are on accumulation number 184 out of 1677
we are on accumulation number 185 out of 1677
we are on accumulation number 186 out of 1677
we are on accumulation number 187 out of 1677
we are on accumulation number 188 out of 1677
we are on accumulation number 189 out of 1677
we are on accumulation number 190 out of 1677
we are on accumulation number 191 out of 1677
we are on accumulation number 192 out of 1677
we are on accumulation number 193 out of 1677
we are on accumulation number 194 out of 1677
we are on accumulation number 195 out of 1677
we are on accumulation number 196 out of 1677
we are on accumulation number 197 out of 1677
we are on accumulation number 198 out of 1677
we are on accumulation number 199 out of 1677
we are on accumulation number 200 out of 1677
we are on accumulation number 201 out of 1677
we are on accumulation number 202 

we are on accumulation number 360 out of 1677
we are on accumulation number 361 out of 1677
we are on accumulation number 362 out of 1677
we are on accumulation number 363 out of 1677
we are on accumulation number 364 out of 1677
we are on accumulation number 365 out of 1677
we are on accumulation number 366 out of 1677
we are on accumulation number 367 out of 1677
we are on accumulation number 368 out of 1677
we are on accumulation number 369 out of 1677
we are on accumulation number 370 out of 1677
we are on accumulation number 371 out of 1677
we are on accumulation number 372 out of 1677
we are on accumulation number 373 out of 1677
we are on accumulation number 374 out of 1677
we are on accumulation number 375 out of 1677
we are on accumulation number 376 out of 1677
we are on accumulation number 377 out of 1677
we are on accumulation number 378 out of 1677
we are on accumulation number 379 out of 1677
we are on accumulation number 380 out of 1677
we are on accumulation number 381 

we are on accumulation number 539 out of 1677
we are on accumulation number 540 out of 1677
we are on accumulation number 541 out of 1677
we are on accumulation number 542 out of 1677
we are on accumulation number 543 out of 1677
we are on accumulation number 544 out of 1677
we are on accumulation number 545 out of 1677
we are on accumulation number 546 out of 1677
we are on accumulation number 547 out of 1677
we are on accumulation number 548 out of 1677
we are on accumulation number 549 out of 1677
we are on accumulation number 550 out of 1677
we are on accumulation number 551 out of 1677
we are on accumulation number 552 out of 1677
we are on accumulation number 553 out of 1677
we are on accumulation number 554 out of 1677
we are on accumulation number 555 out of 1677
we are on accumulation number 556 out of 1677
we are on accumulation number 557 out of 1677
we are on accumulation number 558 out of 1677
we are on accumulation number 559 out of 1677
we are on accumulation number 560 

we are on accumulation number 718 out of 1677
we are on accumulation number 719 out of 1677
we are on accumulation number 720 out of 1677
we are on accumulation number 721 out of 1677
we are on accumulation number 722 out of 1677
we are on accumulation number 723 out of 1677
we are on accumulation number 724 out of 1677
we are on accumulation number 725 out of 1677
we are on accumulation number 726 out of 1677
we are on accumulation number 727 out of 1677
we are on accumulation number 728 out of 1677
we are on accumulation number 729 out of 1677
we are on accumulation number 730 out of 1677
we are on accumulation number 731 out of 1677
we are on accumulation number 732 out of 1677
we are on accumulation number 733 out of 1677
we are on accumulation number 734 out of 1677
we are on accumulation number 735 out of 1677
we are on accumulation number 736 out of 1677
we are on accumulation number 737 out of 1677
we are on accumulation number 738 out of 1677
we are on accumulation number 739 

we are on accumulation number 897 out of 1677
we are on accumulation number 898 out of 1677
we are on accumulation number 899 out of 1677
we are on accumulation number 900 out of 1677
we are on accumulation number 901 out of 1677
we are on accumulation number 902 out of 1677
we are on accumulation number 903 out of 1677
we are on accumulation number 904 out of 1677
we are on accumulation number 905 out of 1677
we are on accumulation number 906 out of 1677
we are on accumulation number 907 out of 1677
we are on accumulation number 908 out of 1677
we are on accumulation number 909 out of 1677
we are on accumulation number 910 out of 1677
we are on accumulation number 911 out of 1677
we are on accumulation number 912 out of 1677
we are on accumulation number 913 out of 1677
we are on accumulation number 914 out of 1677
we are on accumulation number 915 out of 1677
we are on accumulation number 916 out of 1677
we are on accumulation number 917 out of 1677
we are on accumulation number 918 

we are on accumulation number 1074 out of 1677
we are on accumulation number 1075 out of 1677
we are on accumulation number 1076 out of 1677
we are on accumulation number 1077 out of 1677
we are on accumulation number 1078 out of 1677
we are on accumulation number 1079 out of 1677
we are on accumulation number 1080 out of 1677
we are on accumulation number 1081 out of 1677
we are on accumulation number 1082 out of 1677
we are on accumulation number 1083 out of 1677
we are on accumulation number 1084 out of 1677
we are on accumulation number 1085 out of 1677
we are on accumulation number 1086 out of 1677
we are on accumulation number 1087 out of 1677
we are on accumulation number 1088 out of 1677
we are on accumulation number 1089 out of 1677
we are on accumulation number 1090 out of 1677
we are on accumulation number 1091 out of 1677
we are on accumulation number 1092 out of 1677
we are on accumulation number 1093 out of 1677
we are on accumulation number 1094 out of 1677
we are on acc

we are on accumulation number 1249 out of 1677
we are on accumulation number 1250 out of 1677
we are on accumulation number 1251 out of 1677
we are on accumulation number 1252 out of 1677
we are on accumulation number 1253 out of 1677
we are on accumulation number 1254 out of 1677
we are on accumulation number 1255 out of 1677
we are on accumulation number 1256 out of 1677
we are on accumulation number 1257 out of 1677
we are on accumulation number 1258 out of 1677
we are on accumulation number 1259 out of 1677
we are on accumulation number 1260 out of 1677
we are on accumulation number 1261 out of 1677
we are on accumulation number 1262 out of 1677
we are on accumulation number 1263 out of 1677
we are on accumulation number 1264 out of 1677
we are on accumulation number 1265 out of 1677
we are on accumulation number 1266 out of 1677
we are on accumulation number 1267 out of 1677
we are on accumulation number 1268 out of 1677
we are on accumulation number 1269 out of 1677
we are on acc

we are on accumulation number 1424 out of 1677
we are on accumulation number 1425 out of 1677
we are on accumulation number 1426 out of 1677
we are on accumulation number 1427 out of 1677
we are on accumulation number 1428 out of 1677
we are on accumulation number 1429 out of 1677
we are on accumulation number 1430 out of 1677
we are on accumulation number 1431 out of 1677
we are on accumulation number 1432 out of 1677
we are on accumulation number 1433 out of 1677
we are on accumulation number 1434 out of 1677
we are on accumulation number 1435 out of 1677
we are on accumulation number 1436 out of 1677
we are on accumulation number 1437 out of 1677
we are on accumulation number 1438 out of 1677
we are on accumulation number 1439 out of 1677
we are on accumulation number 1440 out of 1677
we are on accumulation number 1441 out of 1677
we are on accumulation number 1442 out of 1677
we are on accumulation number 1443 out of 1677
we are on accumulation number 1444 out of 1677
we are on acc

we are on accumulation number 1599 out of 1677
we are on accumulation number 1600 out of 1677
we are on accumulation number 1601 out of 1677
we are on accumulation number 1602 out of 1677
we are on accumulation number 1603 out of 1677
we are on accumulation number 1604 out of 1677
we are on accumulation number 1605 out of 1677
we are on accumulation number 1606 out of 1677
we are on accumulation number 1607 out of 1677
we are on accumulation number 1608 out of 1677
we are on accumulation number 1609 out of 1677
we are on accumulation number 1610 out of 1677
we are on accumulation number 1611 out of 1677
we are on accumulation number 1612 out of 1677
we are on accumulation number 1613 out of 1677
we are on accumulation number 1614 out of 1677
we are on accumulation number 1615 out of 1677
we are on accumulation number 1616 out of 1677
we are on accumulation number 1617 out of 1677
we are on accumulation number 1618 out of 1677
we are on accumulation number 1619 out of 1677
we are on acc

In [9]:
file = open('hoh_spec_sim_chan_100_100_sec.csv', 'w')
writer = csv.writer(file)
writer.writerow(accum_data)

<function TextIOWrapper.close()>

In [10]:
file.close()

In [ ]:
`